In [34]:
import pandas as pd
import numpy as np
import os
from datetime import datetime
import re #regex \. - dot, \d - digit, \D -non digit, \w - alphanumeric, \W - non alphanumeric

src_dir = r'D:\Ravi_Data_Engineer\02_Target\01_raw'
tgt_dir = r'D:\Ravi_Data_Engineer\02_Target\02_cleansed'
os.makedirs(tgt_dir, exist_ok=True)

def clean_datefields(df, output_format='%Y-%m-%d'):
    for col in df.columns:
        if "date" in col.lower():
            try:
                df[col] = pd.to_datetime(df[col], errors='coerce',dayfirst=True)  # Convert to datetime, coerce errors to NaT       
                df[col] = df[col].dt.strftime(output_format) #Normalize to desired string format (e.g., YYYY-MM-DD)
                
            except Exception:
                pass 

    return df

def clean_products(df,group_col='Category'):
    numeric_cols = df.select_dtypes(include='number').columns
    for col in numeric_cols:
        df[col] = df.groupby(group_col)[col].transform(lambda x: x.fillna(x.mean()))  # fill NaN with mean of the column group by category
        df[col] = df[col].apply(lambda x: round(x, 2))

    return df

def clean_transactions(df):
    numeric_cols = df.select_dtypes(include='number').columns
    df[numeric_cols] = df[numeric_cols].fillna(0)  # fill NaN with mean of the column group by transaction type
    for col in numeric_cols:
        df[col] = df[col].apply(lambda x: round(x, 2))  # round numeric values to 2 decimal places

    return df

keyword = input("Enter the keyword to search in file names:").lower()

files_list_tgt = [f for f in os.listdir(tgt_dir) if keyword in f.lower()]

if files_list_tgt:
    os.remove(os.path.join(tgt_dir, files_list_tgt[0])) # remove the file if it exists in target directory
    print(f"File {os.path.join(tgt_dir, files_list_tgt[0])} deleted from {tgt_dir}.")

files_list_src = [f for f in os.listdir(src_dir) if keyword in f.lower()]

if not files_list_src:
    print(f"No files found containing the keyword '{keyword}' in {src_dir}.")

cleaned_dfs = []

for file in files_list_src:
    file_path = os.path.join(src_dir, file)
    print(f"\n Reading: {file_path}")

    try:
        df = pd.read_csv(file_path) # read csv file

        df.columns = df.columns.str.strip() # remove leading and trailing spaces from column names

        df.columns = df.columns.str.replace(r'\.\d+$', '', regex=True) # replace the last occurrence of a dot followed by digits in column names
        df = df.loc[:,~df.columns.duplicated()] # remove duplicate columns

        df = df.dropna(how='all') # remove rows where all elements are NaN

        string_cols = df.select_dtypes(include='object').columns # checkign the columns with string data type
        df[string_cols] = df[string_cols].fillna('Unknown') # replacing the nulls with Unknown

        numeric_cols = df.select_dtypes(include='number').columns
        df[numeric_cols] = df[numeric_cols].apply(lambda x: round(x, 2)) # rounding numeric values to 2 decimal places

        df = df.map(lambda x: x.strip() if isinstance(x, str) else x) # strip leading and trailing spaces from string values in dataframe
        df = df.map(lambda x: re.sub(r'[^\w\s]', '', str(x)) if isinstance(x, str) else x) # removing special chars from strings

        df = df.drop_duplicates() # drop duplicates

        #df['Phone'] = df['Phone'].apply(lambda x: re.sub(r'\D', '', str(x)) if pd.notnull(x) and len(re.sub(r'\D', '', str(x))) == 10 else '1234567890')
        #using lambda, regex to replace find the Phone field and replace per the needed format

        df = clean_datefields(df)

        if keyword == 'products':
            df = clean_products(df) # assuming clean_products is a function defined elsewhere to clean product related data
        else:
            pass

        if keyword == 'transactions':
            df = clean_transactions(df) # assuming clean_transactions is a function defined elsewhere to clean transaction related data
        else:
            pass

        cleaned_dfs.append((file, df)) # appended to the cleaned_dfs list
        print(f"Cleaned: {file} - {df.shape[0]} rows, {df.shape[1]} columns")

    except Exception as e:
        print(f"Failed to read {file}: {e}")
        continue

for files in cleaned_dfs:
    file_name, df = files
    src_file = os.path.join(src_dir,file_name)
    orig_name = os.path.splitext(file_name)[0]
    ext = os.path.splitext(file_name)[1]
    fn = orig_name.replace('raw_', f'cleansed_')
    new_name = f"{fn}{ext}" # created a new name for the files with prefix as cleansed_
    tgt_file = os.path.join(tgt_dir,new_name)
      
    try:
        df.to_csv(tgt_file, index=False)
        print(f"Saved cleaned data to: {tgt_file}")

    except Exception as e:
        print(f"Failed to save {file_name}: {e}")




 Reading: D:\Ravi_Data_Engineer\02_Target\01_raw\raw_retail_products.csv


C:\Users\K.PRAVEEN\AppData\Local\Temp\ipykernel_23764\632172567.py:15: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df[col] = pd.to_datetime(df[col], errors='coerce',dayfirst=True)  # Convert to datetime, coerce errors to NaT


Cleaned: raw_retail_products.csv - 500 rows, 6 columns
Saved cleaned data to: D:\Ravi_Data_Engineer\02_Target\02_cleansed\cleansed_retail_products.csv
